In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina' # very useful command for high-res images

# Imports

In [2]:
# ALL IMPORTS
import matplotlib.pyplot as plt
import matplotlib.style
import matplotlib as mpl
import pandas as pd
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages

import os
from os.path import isfile, join
from os import walk

In [3]:
mpl.style.use("classic")

# BULKS by 3 rows on PDF

In [4]:
#Load all directories from all files into lists

fraw = []
fCBV = []
fOLC = [] #Outliner Cleaned

for (dirpath, dirnames, filenames) in walk("..\\_TESS_lightcurves_raw"):
    for filename in filenames:
        full_path = os.path.join(dirpath, filename)
        pathTittle = full_path.split(os.sep)
        step = []
        step.append(full_path)
        step.append(pathTittle[-2])
        step.append(pathTittle[-1])
        fraw.append(step)

for (dirpath, dirnames, filenames) in walk("..\\_TESS_lightcurves_median_after_detrended"):
    for filename in filenames:
        full_path = os.path.join(dirpath, filename)
        pathTittle = full_path.split(os.sep)
        step = []
        step.append(full_path)
        step.append(pathTittle[-2])
        step.append(pathTittle[-1])
        fCBV.append(step)

for (dirpath, dirnames, filenames) in walk("..\\_TESS_lightcurves_outliercleaned"):
    for filename in filenames:
        full_path = os.path.join(dirpath, filename)
        pathTittle = full_path.split(os.sep)
        step = []
        step.append(full_path)
        step.append(pathTittle[-2])
        step.append(pathTittle[-1])
        fOLC.append(step)

In [5]:
# Create categories to separate PDFs

categories = []
for i in range(len(fraw)):
    if fraw[i][1] not in categories:
        categories.append(fraw[i][1])

In [6]:
print(categories)

['ACV', 'CEP', 'DCEP', 'DCEP-FU', 'DCEPS', 'DSCT', 'E', 'EA', 'EB', 'EC', 'ED', 'EW', 'HADS', 'L', 'ROT', 'RR', 'RRAB', 'RRAB_BL', 'RRC', 'RRD', 'RS', 'SR']


In [30]:
def plotting3(lc_data, lc_dataCBV=None, lc_dataOLC=None, tittle="", axs=None):
    tess_data_raw = lc_data.to_records(index=False)
    tess_data_CBV = lc_dataCBV.to_records(index=False)
    tess_data_OLC = lc_dataOLC.to_records(index=False)

    # First Graph: Light Curve Raw
    axs[0].errorbar(tess_data_raw["JD"], tess_data_raw["MAG"], yerr=tess_data_raw["ERR"], fmt=".k", ecolor="gray", label=r"Light Curve")
    axs[0].grid(which="major")
    axs[0].legend(loc="lower left")
    axs[0].minorticks_on()
    axs[0].invert_yaxis()
    axs[0].set_title(f"{tittle} - Raw Data")
    axs[0].tick_params(which='both', direction='in', tick2On=True)
    axs[0].ticklabel_format(useOffset=False)

    # Second Graph: Light Curve after CBV
    axs[1].errorbar(tess_data_CBV["JD"], tess_data_CBV["MAG_Clean"], yerr=tess_data_CBV["ERR"], fmt=".k", ecolor="gray", label=r"Light Curve (After CBV)")
    axs[1].grid(which="major")
    axs[1].legend(loc="lower left")
    axs[1].minorticks_on()
    axs[1].invert_yaxis()
    axs[1].set_title(f"{tittle} - After CBV")
    axs[1].tick_params(which='both', direction='in', tick2On=True)
    axs[1].ticklabel_format(useOffset=False)

    # 3rd Graph: Light Curve after outlier cleaned
    axs[2].errorbar(tess_data_OLC["JD"], tess_data_OLC["MAG"], yerr=tess_data_OLC["ERR"], fmt=".k", ecolor="gray", label=r"Light Curve Outlier Cleaned")
    axs[2].grid(which="major")
    axs[2].legend(loc="lower left")
    axs[2].minorticks_on()
    axs[2].invert_yaxis()
    axs[2].set_title(f"{tittle} - After OLC")
    axs[2].tick_params(which='both', direction='in', tick2On=True)
    axs[2].ticklabel_format(useOffset=False)

In [ ]:
#Testing

for i in range(2):
    data_raw = pd.read_csv(fraw[i][0], delim_whitespace=True, names=["JD", "MAG", "ERR"], na_values="*********")
    data_CBV = pd.read_csv(fCBV[i][0], delim_whitespace=True, names=["JD", "MAG_Clean", "MAG_After_CBV", "ERR"], na_values="*********")
    data_OLC = pd.read_csv(fOLC[i][0], delimiter=",", names=["JD", "MAG", "ERR"], na_values="*********")
    title = (fraw[i][1] + " " + fraw[i][2])[:-3]
    plotting3(data_raw,data_CBV,data_OLC, title)

In [ ]:
#Plotting and Save on PDF

i = 0
j = 0
while j < len(categories):
    while i < len(fraw) and fraw[i][1] == categories[j]:
        print(i, j) # Archive Index, Category Index
        data_raw = pd.read_csv(fraw[i][0], delim_whitespace=True, names=["JD", "MAG", "ERR"], na_values="*********")
        data_CBV = pd.read_csv(fCBV[i][0], delim_whitespace=True, names=["JD", "MAG_Clean", "MAG_After_CBV", "ERR"], na_values="*********")
        data_OLC = pd.read_csv(fOLC[i][0], delimiter=",", names=["JD", "MAG", "ERR"], na_values="*********")
        fig, axs = plt.subplots(3, 1, figsize=(10, 12))  # 3 rows y 1 column
        fig.subplots_adjust(hspace=0.3)
        title = (fraw[i][1] + " " + fraw[i][2])[:-3]
        plotting3(data_raw,data_CBV,data_OLC, title, axs)
        subpath = categories[j]+"_LightCurves"
        file = fraw[i][2][:-3] + ".pdf"
        os.makedirs(subpath, exist_ok=True)
        completePath = os.path.join(subpath, file)
        with PdfPages(completePath) as pdf:
            pdf.savefig()
        plt.close()
        i += 1
    j += 1

22
0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 0
15 0
16 0
17 0
18 0
19 0
20 0
21 1
22 1
23 1
24 1
25 1
26 1
27 1
28 1
29 1
30 1
31 1
32 1
33 1
34 1
35 1
36 1
37 1
38 1
39 1
40 1
41 1
42 1
43 2
44 2
45 2
46 2
47 2
48 2
49 2
50 2
51 2
52 2
53 2
54 2
55 2
56 2
57 2
58 2
59 2
60 2
61 2
62 2
63 2
64 2
65 2
66 2
67 2
68 2
69 2
70 2
71 2
72 2
73 2
74 2
75 2
76 2
77 2
78 2
79 2
80 2
81 2
82 2
83 2
84 2
85 2
86 2
87 2
88 2
89 2
90 2
91 2
92 2
93 2
94 2
95 2
96 2
97 2
98 2
99 2
100 2
101 2
102 2
103 2
104 2
105 2
106 2
107 2
108 2
109 2
110 2
111 2
112 2
113 2
114 2
115 2
116 2
117 2
118 2
119 2
120 3
121 3
122 3
123 3
124 3
125 3
126 3
127 3
128 3
129 3
130 4
131 4
132 4
133 4
134 4
135 4
136 5
137 5
138 5
139 5
140 5
141 5
142 5
143 5
144 5
145 5
146 5
147 5
148 5
149 5
150 5
151 5
152 5
153 5
154 5
155 5
156 5
157 5
158 5
159 5
160 5
161 5
162 5
163 5
164 5
165 5
166 5
167 5
168 5
169 5
170 5
171 5
172 5
173 5
174 5
175 5
176 5
177 5
178 5
179 5
180 5
181 5
182 5
183 5
184